In [9]:
import base64
import requests
import time
import pandas as pd
import os

# OpenAI API Key - Replace with your GPT API key
api_key = ""

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Read CSV
questions_df = pd.read_csv("C:/Users/amitc/OneDrive/Desktop/New folder (7)/VLAT/VLAT Questions.csv")
questions = questions_df.to_dict('records')
responses_data = []


for i, question in enumerate(questions, start=1):
    time.sleep(5)
    try:
        print(f"\nProcessing question {i}:")
        print(question)
        
        # Path to your image
        image_path = "C:/Users/amitc/OneDrive/Desktop/New folder (7)/VLAT/Images/" + str(question.get('vis', '')) + ".png"
        
        # Question text
        question_text = question.get('question: ', '')
        question_options = question.get('option:', '')
        correct_ans = str(question.get('correct', '')).strip()
        
        print(f"Processing image: {image_path}")
        print(f"Question: {question_text}")
        print(f"Options: {question_options}")
        print(f"Correct answer: {correct_ans}")
        
        # Getting the base64 string
        base64_image = encode_image(image_path)
        
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {api_key}"
        }
        
        payload = {
            "model": "gpt-4o-2024-11-20", # Use GPT-4o for vision capabilities
            "max_tokens": 5000,
            "temperature": 0.0,
            "messages": [
                {
                    "role": "system",
                    "content": "You are an assistant, skilled in reading and interpreting visually represented data."
                },
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": 'I am about to show you a graph and ask you a multiple-choice question about that graph. \n\n' +
                                   'Task 1: Data Extraction and Table Creation: First, explicitly list ALL numerical values you can identify on both axes, then create a structured table using markdown syntax that includes ALL data points you identified above with appropriate column headers with units. \n \n' +
                                   'Task 2: Sort the data: Sort the data in descending order by the numerical values. \n \n' +
                                   'Task 3: Data Verification and Error Handling: Double-check if your table matches ALL elements in the graph by comparing each value in your table with the graph and updating your table with correct values, verify the sorting is correct, and before proceeding, confirm all corrections have been made and use ONLY the corrected data for analysis. \n \n' +
                                   'Task 4: Question Analysis: Using ONLY the verified data in your table, compare EACH value individually with the reference value, for "less than" comparisons mark ALL values that are even slightly below the reference, for "greater than" comparisons mark ALL values that are even slightly above the reference, and show each comparison on a new line. \n \n' +
                                   'Provide your reasoning with specific references to table values. \n\n' +
                                   'End with: "Correct Answer: "' +
                                   'Let\'s solve this step by step.' +
                                   '\n\n' + question_text + " " + question_options
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/png;base64,{base64_image}"
                            }
                        }
                    ]
                }
            ]
        }
        
        # Add retry logic for overloaded errors
        max_retries = 3
        retry_delay = 20  # seconds
        
        for retry in range(max_retries):
            try:
                time_start = time.perf_counter()
                response_raw = requests.post(
                    "https://api.openai.com/v1/chat/completions",
                    headers=headers,
                    json=payload
                )
                time_end = time.perf_counter()
                
                response = response_raw.json()
                
                # If we get a rate limit error, wait and retry
                if 'error' in response and response['error'].get('code') == 'rate_limit_exceeded':
                    if retry < max_retries - 1:  # Don't sleep on the last retry
                        print(f"\nAPI rate limited, waiting {retry_delay} seconds before retry {retry + 1}/{max_retries}")
                        time.sleep(retry_delay)
                        continue
                
                break  # If we get here, we either got a good response or a different error
                
            except Exception as e:
                print(f"Error during API call (attempt {retry + 1}/{max_retries}):", str(e))
                if retry < max_retries - 1:
                    time.sleep(retry_delay)
                    continue
                raise  # Re-raise the last exception if we've exhausted all retries
        print("\nAPI Response:", response)  # Debug print
        
        if 'error' in response:
            gpt_answer = f"Error: {response['error'].get('message', 'Unknown error')}"
            is_correct = "N/A"
        else:
            # Extract the answer after "Correct Answer: "
            full_response = response["choices"][0]["message"]["content"].strip()
            if "Correct Answer: " in full_response:
                gpt_answer = full_response.split("Correct Answer: ")[-1].strip()
                # Case-insensitive comparison after stripping whitespace
                is_correct = gpt_answer.strip().upper() == correct_ans.strip().upper()
            else:
                gpt_answer = "Error: No answer in correct format"
                is_correct = "N/A"
        
        responses_data.append([gpt_answer, time_end-time_start, is_correct])
        print(f"\nAnswer: {gpt_answer}")
        print(f"Time taken: {time_end-time_start:.2f} seconds")
        print(f"Correct? {is_correct}")
        
        # Increase delay between requests to 15 seconds
        time.sleep(max(15 - (time_end-time_start), 0))
        
    except Exception as e:
        print(f"Error processing question {i}:", str(e))
        responses_data.append([f"Error: {str(e)}", 0, "N/A"])

# Create Results directory if it doesn't exist
results_dir = "C:/Users/amitc/My Drive/10_Ph.D. at SBU/4_2025/3_LLM Comparison/VLAT/1_GPT 4.o_Final Results with Explanation/"
os.makedirs(results_dir, exist_ok=True)

results_df = pd.DataFrame(responses_data, columns=['response', 'time', 'correct_bool'])
results_df.index = range(1, results_df.shape[0] + 1)
results_df.to_csv(results_dir + "GPT_VLAT_" + str(int(time.time())) + ".csv", index_label="id")
print("\n*** Finished ***")


Processing question 1:
{'id': 1, 'dropped': 'no', 'vis': 'VLAT_a', 'item': 'a_1', 'question: ': 'What was the price of a barrel of oil in February 2015? ', 'option:': '$57.36; $47.82; $50.24; $39.72', 'correct': '$50.24 '}
Processing image: C:/Users/amitc/OneDrive/Desktop/New folder (7)/VLAT/Images/VLAT_a.png
Question: What was the price of a barrel of oil in February 2015? 
Options: $57.36; $47.82; $50.24; $39.72
Correct answer: $50.24

API Response: {'id': 'chatcmpl-BG905xrW2YVP8t4yoaMaKAJpkJb1u', 'object': 'chat.completion', 'created': 1743187545, 'model': 'gpt-4o-2024-11-20', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '### Task 1: Data Extraction and Table Creation\n\n#### Numerical Values on Axes:\n- **X-axis (Months):** January, February, March, April, May, June, July, August, September, October, November, December.\n- **Y-axis (Oil Price in $ per barrel):** 35, 40, 45, 50, 55, 60, 65.\n\n#### Data Points from the Graph:\n| Month      | Oil Price ($/bar


Processing question 4:
{'id': 4, 'dropped': 'no', 'vis': 'VLAT_a', 'item': 'a_4', 'question: ': 'Over the course of the second half of 2015, the price of a barrel of oil was ____________.', 'option:': 'rising; falling; staying', 'correct': 'falling'}
Processing image: C:/Users/amitc/OneDrive/Desktop/New folder (7)/VLAT/Images/VLAT_a.png
Question: Over the course of the second half of 2015, the price of a barrel of oil was ____________.
Options: rising; falling; staying
Correct answer: falling

API Response: {'id': 'chatcmpl-BG913IOgHnFTrYRkNJ5JnrG5VV4ID', 'object': 'chat.completion', 'created': 1743187605, 'model': 'gpt-4o-2024-11-20', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '### Task 1: Data Extraction and Table Creation\n\n#### Numerical Values from the Graph:\n- **X-axis (Months):** January, February, March, April, May, June, July, August, September, October, November, December\n- **Y-axis (Oil Price in $ per barrel):** 35, 40, 45, 50, 55, 60, 65\n\n###


Processing question 7:
{'id': 7, 'dropped': 'no', 'vis': 'VLAT_b', 'item': 'b_2', 'question: ': 'In which country is the average internet speed the fastest in Asia?', 'option:': 'China; Hong Kong; South Korea; Vietnam', 'correct': 'South Korea'}
Processing image: C:/Users/amitc/OneDrive/Desktop/New folder (7)/VLAT/Images/VLAT_b.png
Question: In which country is the average internet speed the fastest in Asia?
Options: China; Hong Kong; South Korea; Vietnam
Correct answer: South Korea

API Response: {'id': 'chatcmpl-BG9214cprmRaLADQSXa5OrGkmCPQ7', 'object': 'chat.completion', 'created': 1743187665, 'model': 'gpt-4o-2024-11-20', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '### Task 1: Data Extraction and Table Creation\n\n#### Numerical Values on Axes:\n- **Speed (Mbps)** (y-axis): 0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22\n- **Countries** (x-axis): Australia, China, Hong Kong, India, Indonesia, Japan, Malaysia, New Zealand, Singapore, South Korea, Sri Lanka, Tai


API Response: {'id': 'chatcmpl-BG92gZHHj84fUdAyaJV95YqREdT9c', 'object': 'chat.completion', 'created': 1743187706, 'model': 'gpt-4o-2024-11-20', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '### Task 1: Data Extraction and Table Creation\n\n#### Numerical Values on Axes:\n- **Y-axis (Speed in Mbps):** 0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22\n- **X-axis (Countries):** Australia, China, Hong Kong, India, Indonesia, Japan, Malaysia, New Zealand, Singapore, South Korea, Sri Lanka, Taiwan, Thailand, Vietnam\n\n#### Data Points:\n| Country       | Speed (Mbps) |\n|---------------|--------------|\n| Australia     | 8            |\n| China         | 4            |\n| Hong Kong     | 16           |\n| India         | 2            |\n| Indonesia     | 3            |\n| Japan         | 14           |\n| Malaysia      | 5            |\n| New Zealand   | 10           |\n| Singapore     | 18           |\n| South Korea   | 20           |\n| Sri Lanka     | 6            |\n|


API Response: {'id': 'chatcmpl-BG93Nb9o90Y3mWOHx26pviNaXoOgG', 'object': 'chat.completion', 'created': 1743187749, 'model': 'gpt-4o-2024-11-20', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '### Task 1: Data Extraction and Table Creation\n\n#### Numerical Values on Axes:\n- **Y-axis (Cost in $):** 0, 10, 20, 30, 40, 50, 60\n- **X-axis (Cities):** New York City, Las Vegas, Honolulu, Atlanta, Boston, Washington D.C., Chicago, Orlando, San Francisco, Seattle\n\n#### Data Points:\nFrom the graph, the costs for each item (Vodka, Soda, Peanuts, Water, Sandwich) in each city are extracted.\n\n| City            | Vodka ($) | Soda ($) | Peanuts ($) | Water ($) | Sandwich ($) | Total Cost ($) |\n|-----------------|-----------|----------|-------------|-----------|---------------|----------------|\n| New York City   | 20        | 10       | 10          | 5         | 15            | 60             |\n| Las Vegas       | 15        | 10       | 10          | 5         | 15   

...


Processing question 53:
{'id': 53, 'dropped': 'no', 'vis': 'VLAT_h', 'item': 'h_7', 'question: ': 'In general, the ridership of the metro system increases as the number of stations increases.', 'option:': 'True; False', 'correct': 'FALSE'}
Processing image: C:/Users/amitc/OneDrive/Desktop/New folder (7)/VLAT/Images/VLAT_h.png
Question: In general, the ridership of the metro system increases as the number of stations increases.
Options: True; False
Correct answer: FALSE

API Response: {'id': 'chatcmpl-BG9Ho3Iu5aNkqyOqDpfKPZGxvduag', 'object': 'chat.completion', 'created': 1743188644, 'model': 'gpt-4o-2024-11-20', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '### Task 1: Data Extraction and Table Creation\n\n#### Numerical Values Identified:\n- **X-axis (Number of Stations):** 100, 150, 200, 250, 300, 350, 400, 450\n- **Y-axis (Total System Length in km):** 150, 200, 250, 300, 350, 400, 450, 500, 550, 600\n- **Ridership (bn per year):** 1.5, 2.5, 3.5 (represented


Processing question 55:
{'id': 55, 'dropped': 'no', 'vis': 'VLAT_k', 'item': 'k_1', 'question: ': 'What was the unemployment rate for Indiana (IN) in 2015?', 'option:': '1.1% - 2.3%; 2.3% - 3.4%; 3.4% - 4.6%; 4.6% - 5.7%', 'correct': '3.4% - 4.6%'}
Processing image: C:/Users/amitc/OneDrive/Desktop/New folder (7)/VLAT/Images/VLAT_k.png
Question: What was the unemployment rate for Indiana (IN) in 2015?
Options: 1.1% - 2.3%; 2.3% - 3.4%; 3.4% - 4.6%; 4.6% - 5.7%
Correct answer: 3.4% - 4.6%

API Response: {'id': 'chatcmpl-BG9IZEYfryAOKM3ClfPR2KgUJFQBh', 'object': 'chat.completion', 'created': 1743188691, 'model': 'gpt-4o-2024-11-20', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "### Task 1: Data Extraction and Table Creation\n\n#### Numerical Values on the Axes:\n- **Legend Values (Unemployment Rate Ranges):**\n  - 0.0% - 1.1%\n  - 1.1% - 2.3%\n  - 2.3% - 3.4%\n  - 3.4% - 4.6%\n  - 4.6% - 5.7%\n  - 5.7% - 6.9%\n\n#### Data Table:\n| State | Unemployment Rate (%) |\


API Response: {'id': 'chatcmpl-BG9JY2f5fQpcOcu4gnQLsy96IHC17', 'object': 'chat.completion', 'created': 1743188752, 'model': 'gpt-4o-2024-11-20', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "### Task 1: Data Extraction and Table Creation\n\n#### Numerical Values Identified:\n- The graph does not explicitly provide numerical values for the number of unique visitors. Instead, the relative sizes of the rectangles represent the number of visitors. \n\n#### Table Creation:\nSince no explicit numerical values are provided, I will create a table listing the categories and subcategories as they appear in the graph.\n\n| **Category**   | **Subcategory**       | **Relative Size** |\n|-----------------|-----------------------|--------------------|\n| Financial       | Citibank             | Small             |\n| Financial       | Chase                | Small             |\n| Financial       | PayPal               | Medium            |\n| Financial       | Experian       


Processing question 60:
{'id': 60, 'dropped': 'no', 'vis': 'VLAT_l', 'item': 'l_3', 'question: ': 'The number of unique visitors for Amazon was more than that of Yahoo in 2010.', 'option:': 'True; False', 'correct': 'FALSE'}
Processing image: C:/Users/amitc/OneDrive/Desktop/New folder (7)/VLAT/Images/VLAT_l.png
Question: The number of unique visitors for Amazon was more than that of Yahoo in 2010.
Options: True; False
Correct answer: FALSE

API Response: {'id': 'chatcmpl-BG9KOPvXQXOq1tu9KYbwInuC1ZGw6', 'object': 'chat.completion', 'created': 1743188804, 'model': 'gpt-4o-2024-11-20', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'To solve this, let\'s proceed step by step.\n\n---\n\n### Task 1: Data Extraction and Table Creation\n\n#### Numerical Values on Axes:\nThe graph does not explicitly provide numerical values on the axes. Instead, the relative sizes of the rectangles represent the number of unique visitors for each website. Therefore, we will extract the 


*** Finished ***
